# Reconstruct and model glycolysis

Building and simulation pathways using COBRApy. 


In [3]:
import cobra
from fba4l_course_material.resources.glycolysis_pathway import metabolites_and_info, reactions_and_info

In [4]:
# The reactions and metabolites to be added in this exercise
try:
    import escher
    builder = escher.Builder(map_json='../resources/glycolysis_map.json')
    builder
except:
    pass

---------
## A) Add the pathway metabolites and their information to a new model

The goal of this exercise will be to build a simulate the glycolysis synthesis pathway shown above. 

This section will introduce the first steps of adding the pathway to model.

### 1) Initialize an empty COBRApy model

Print the number of metabolites and reactions in the model

### 2) Add metabolites and info to the model

**Hints:**

1) The information regarding the reactions and metabolites involved in this pathway can be found in `reactions_and_info` and `metabolites_and_info`, respectively. These are imported above.
  - These are imported in the form of a **dictionary**
  - Print the contents of `metabolites_and_info`

----

- A dictionary is a Python data structure that consists of both `keys` and `values`. This allows you to look up quantities based on the `key` you are interested in. An example of this is shown below for ATP

----

In [ ]:
# Look up the information assocated with the histidine metabolite (his__L_c)
print('Values associated with atp_c = ', metabolites_and_info['atp_c'])
# This returns another dictionary where the formula, etc. can be looked up with:
print('Formula = ', metabolites_and_info['atp_c']['formula'])

2) To leverage dictionary properties to add the metabolites to model more efficiently, iterate through all of the dictionary entries using the `items` method and print the `keys` and `values`

3) Add the metabolites, along with the metabolite information to the empty model.
 - Then print the number of metabolites in the model 

-------
## B) Add reactions to model and simulate

### 1) Like above, add the reaction info found in `reactions_and_info` to the model
 

### 2)  Mass balance check each reaction added to the pathway using `reaction.check_mass_balance()` method

This is an important step in quality controlling any metabolic model. Failing to do so could result in incorrect reactions and incorrect predictions


### 3) If a reaction is not mass balanced, correct it with `reaction.add_metabolites()` and confirm it is now mass balance
The proper stoichiometry for the reaction can be found at [ecocyc](http://www.ecocyc.com) or [bigg](http://bigg.ucsd.edu). Databases like these are useful references for metabolic reconstructions and understanding models.

**Hint:**
1) Metabolites can be add and removed from reactions using `add_metabolites` on an existing reaction like below

### 4) Add boundary reactions to act as source and sink for glucose and pyruvate

Models require inputs and outputs in order to simulate. In other words, flux going into a metabolite must ultimately go somewhere.

The reactions should have the following info:

| ID      | Reaction       |   Lower Bound | Upper Bound 
| :-------------: |:-------------:|:-------------:|:-------------:|
| EX_glc__D_c | glc__D_c <=>     | -1 | 1000 |
| EX_pyr_c | pyr_c -->     | 0 | 1000 |
| EX_h2o_c | h2o_c <=>     | -1000 | 1000 |
| EX_h_c | h_c <=>     | -1000 | 1000 |
| EX_pi_c | pi_c <=>     | -1000 | 1000 |

These reactions are called **Exchange Reactions** and by convention begin with 'EX_'. Further, by convention a negative lower bound/flux corresponds to a model input (or metabolite uptake from the growth media/growth condition). Therefore, a lower bound of 1 means that the model can take up, at most, 1 $\frac{mmol}{gDW \cdot hr}$ of glucose

### 5) View the growth media of the model based on the exchange reactions using `model.medium`

### 6) Set the model objective to pyruvate production ('EX_pyr_e') and optimize

The objective corresponds to the reaction flux that the optimization will either maximize or minimize (by default maximize).

**Still no flux through the reaction?  Why?**

Flux balance analysis assumes that the system is opperating at steady state (meaning the concentration of the metabolites in the system do not increase or decrease over time). As a consequence of this, each metabolite that participates in the pathway must be created and consumed at the same rate. 


ATP and NADH are generated in glycolysis but not used
- Add ATPM and DM_nadh reactions from bigg

### 7) Summarzing the metabolic fate of pyruvate / NADH
Pyruvate and NADH facillitate metabolic processes in pathways we will model later.

For now, summarize the metabolic fates of these two metabolites with the following reactions (after adding a reaction to the model, you can use the reactions `build_reaction_from_sring` method):

- **NADH_to_ATP**: nadh_c+ .5 o2_c + 3.5 h_c + 2.5 pi_c + 2.5 adp_c -> nad_c + 3.5 h2o_c + 2.5 atp_c
  - 2.5 ATP per NADH (electron transport chain)
- **pyr_to_ATP**: pyr_c + 3 h2o_c + 12.5 h_c + 12.5 pi_c + 12.5 adp_c -> 3 co2_c + 12.5 h2o_c + 12.5 atp_c + 9 h_c
  - 15 ATP per pyruvate (TCA cycle + electron transport chain)
  
  
**Note**: also need to remove the DM_nadh reaction
- Can use reaction's `remove_from_model` method 

#### Note that you need to add `o2_c` and `co2_c` metabolites into the model
These therefore need exchange reactions into and out of the system
- Use upper and lower bound of 1000 and -1000, respectively

#### Set objective to `ATPM` and optimize

### 8) Modeling the oxygen dependance of ATP production

1. Set objective to ATPM
2. Set the lower bound of `EX_o2_c` to 50 values ranging from -2 to 0
  - Can use numpy's `linspace` function
3. Optimize and print the ATPM values